In [1]:
from uniswappy import *
import time

sell_token = '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48'
buy_token = '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2'
sell_amount = '10000000'

api = API0x(chain = Chain0x.ETHEREUM)
res = api.apply(sell_token, buy_token, sell_amount)

In [2]:
prices = []

USER_NM = 'user0'
eth_amount = 100
tkn_amount = eth_amount/float(res['price'])

usdc = ERC20("USDC", "0x111")
eth = ERC20("ETH", "0x09")
exchg_data = UniswapExchangeData(tkn0 = eth, tkn1 = usdc, symbol="LP", address="0x011")

factory = UniswapFactory("ETH pool factory", "0x2")
lp = factory.deploy(exchg_data)
lp.add_liquidity("user0", eth_amount, tkn_amount, eth_amount, tkn_amount)
lp.summary()


Exchange ETH-USDC (LP)
Reserves: ETH = 100, USDC = 307483.4675175299
Liquidity: 5545.119182826731 



In [3]:
N = 5
tkn_price_arr = []

for k in range(N):
    res = api.apply(sell_token, buy_token, sell_amount)
    time.sleep(3) 
    tkn_price_arr.append(1/float(res['price']))
    print(f"price: {1/float(res['price'])}")

price: 3074.8346751752993
price: 3074.8346751752993
price: 3074.8346751752993
price: 3074.8346751752993
price: 3074.8346751752993


In [4]:
lp_tot_arr = []
x_amt_arr = []
y_amt_arr = []
lp_price_arr = []
correction = CorrectReserves(lp, x0 = tkn_price_arr[0])
tDel = TokenDeltaModel(50)

for p in tkn_price_arr[1:]: 
    
    # arbitrage correction
    correction.apply(p)
    lp_price_arr.append(lp.get_price(correction.get_x_tkn()))  
    x_amt_arr.append(lp.get_reserve(correction.get_y_tkn()))  
    y_amt_arr.append(lp.get_reserve(correction.get_x_tkn()))  
    
    # take a random trade
    out = RandomSwap(tDel = tDel).apply(lp, eth, USER_NM)
    lp_price_arr.append(lp.get_price(correction.get_x_tkn()))  
    x_amt_arr.append(lp.get_reserve(correction.get_y_tkn()))  
    y_amt_arr.append(lp.get_reserve(correction.get_x_tkn()))        

In [5]:
x_amt_arr

[307483.4675175291,
 307492.90438117756,
 307487.2341197242,
 307499.3733065028,
 307488.2115258657,
 307491.93732460676,
 307488.21713700093,
 307511.0259053429]

In [6]:
y_amt_arr

[100.00000000000028,
 99.99694023706277,
 100.00184874852089,
 99.99791280736066,
 100.00154284988852,
 100.00033479259011,
 100.00154467474603,
 99.99414959084798]

In [7]:
lp_price_arr

[3074.834675175282,
 3075.023132229887,
 3074.8154955912473,
 3075.057915447495,
 3074.834675175299,
 3074.9090786783195,
 3074.8346751752997,
 3075.290176111343]